ideas:
<br>
- clustering gradients and band percentages
<br>
- situation 1: equilibrium, (upper - lower) / lower *100 < x=10
<br>
- situation 2: bullish, (upper-lower) / lower *100 > x=10 && price > middle
<br>
- situation 3: bearish, (upper - lower) / lower *100 > x=10 && price < middle
<br>
- plot return of situation's group of candles


In [1]:
#making necessary imports and configuring APIs
import pandas as pd
import ta
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from mpl_toolkits import mplot3d
from dotenv import load_dotenv
from binance.client import Client
import csv
import os

load_dotenv()
b_key = os.getenv("BinanceAPI_Key")
b_secret = os.getenv("BinanceAPI_Secret")
client = Client(b_key,b_secret)


In [2]:
df = yf.download("AAPL",start="2017-01-01")
df = df[["Open","High","Low","Close"]]

[*********************100%***********************]  1 of 1 completed


In [3]:
#to-do: fetch btc price data for all intervals
#daily in 3 month intervals
#4 hours in 1 month intervals
#1 hour in weekly intervals
#15 minute in 3 day intervals
#5 min
#1 min

def fetch_daily_binance_data():
    candles = client.get_historical_klines("BTCUSDT",Client.KLINE_INTERVAL_1DAY,"1 Jan, 2022", "1 July, 2022")
    col = ["Date", "Open", "High", "Low", "Close"]
    ret_df = pd.DataFrame(candles)
    ret_df = ret_df[[0,1,2,3,4]]
    ret_df.columns = col
    ret_df["Date"] = pd.to_datetime(ret_df["Date"], unit="ms")
    ret_df.set_index("Date",inplace=True)
    ret_df = ret_df.astype(float)
    return ret_df
def fetch_fourhour_binance_data():
    candles = client.get_historical_klines("BTCUSDT",Client.KLINE_INTERVAL_4HOUR,"1 Jan, 2022", "1 July, 2022")
    col = ["Date", "Open", "High", "Low", "Close"]
    ret_df = pd.DataFrame(candles)
    ret_df = ret_df[[0,1,2,3,4]]
    ret_df.columns = col
    ret_df["Date"] = pd.to_datetime(ret_df["Date"], unit="ms")
    ret_df.set_index("Date",inplace=True)
    ret_df = ret_df.astype(float)
    return ret_df
def fetch_1hour_binance_data():
    candles = client.get_historical_klines("BTCUSDT",Client.KLINE_INTERVAL_1HOUR,"1 Jan, 2022", "1 July, 2022")
    col = ["Date", "Open", "High", "Low", "Close"]
    ret_df = pd.DataFrame(candles)
    ret_df = ret_df[[0,1,2,3,4]]
    ret_df.columns = col
    ret_df["Date"] = pd.to_datetime(ret_df["Date"], unit="ms")
    ret_df.set_index("Date",inplace=True)
    ret_df = ret_df.astype(float)
    return ret_df
def fetch_15min_binance_data():
    candles = client.get_historical_klines("BTCUSDT",Client.KLINE_INTERVAL_15MINUTE,"1 Jan, 2022", "1 July, 2022")
    col = ["Date", "Open", "High", "Low", "Close"]
    ret_df = pd.DataFrame(candles)
    ret_df = ret_df[[0,1,2,3,4]]
    ret_df.columns = col
    ret_df["Date"] = pd.to_datetime(ret_df["Date"], unit="ms")
    ret_df.set_index("Date",inplace=True)
    ret_df = ret_df.astype(float)
    return ret_df
def fetch_5min_binance_data():
    candles = client.get_historical_klines("BTCUSDT",Client.KLINE_INTERVAL_5MINUTE,"1 Jan, 2022", "1 July, 2022")
    col = ["Date", "Open", "High", "Low", "Close"]
    ret_df = pd.DataFrame(candles)
    ret_df = ret_df[[0,1,2,3,4]]
    ret_df.columns = col
    ret_df["Date"] = pd.to_datetime(ret_df["Date"], unit="ms")
    ret_df.set_index("Date",inplace=True)
    ret_df = ret_df.astype(float)
    return ret_df
def fetch_1min_binance_data():
    candles = client.get_historical_klines("BTCUSDT",Client.KLINE_INTERVAL_1MINUTE,"1 Jan, 2022", "1 July, 2022")
    col = ["Date", "Open", "High", "Low", "Close"]
    ret_df = pd.DataFrame(candles)
    ret_df = ret_df[[0,1,2,3,4]]
    ret_df.columns = col
    ret_df["Date"] = pd.to_datetime(ret_df["Date"], unit="ms")
    ret_df.set_index("Date",inplace=True)
    ret_df = ret_df.astype(float)
    return ret_df
#fetch_daily_binance_data()

,Open,High,Low,Close
Date,,,,
2022-01-01,46216.93,47954.63,46208.37,47722.65
2022-01-02,47722.66,47990.00,46654.00,47286.18
2022-01-03,47286.18,47570.00,45696.00,46446.10
2022-01-04,46446.10,47557.54,45500.00,45832.01
2022-01-05,45832.01,47070.00,42500.00,43451.13
...,...,...,...,...
2022-06-27,21038.08,21539.85,20510.00,20742.56
2022-06-28,20742.57,21212.10,20202.01,20281.29
2022-06-29,20281.28,20432.31,19854.92,20123.01


In [15]:
#adding bollinger indicators to df
def bollinger(df):
    df["bollinger high band"] = ta.volatility.bollinger_hband(df.Close)
    df["bollinger low band"] = ta.volatility.bollinger_lband(df.Close)
    df["bollinger middle band"] = ta.volatility.bollinger_mavg(df.Close)
    df["band width"] = ta.volatility.bollinger_wband(df.Close)
    #adding high and low band slope into data frame
    high_grad_array = np.gradient(df["bollinger high band"])
    low_grad_array = np.gradient(df["bollinger low band"])
    df["bollinger high gradient"] = high_grad_array
    df["bollinger low gradient"] = low_grad_array
    #adding bollinger band percentage into dataframe
    df["bollinger band percentage"] = (df["bollinger high band"] - df["bollinger low band"]) / df["bollinger low band"] *100

In [16]:
bollinger(df)

In [6]:
#converting dataframe into a csv file
df.to_csv("test.csv")

In [17]:
df

,Open,High,Low,Close,bollinger high band,bollinger low band,bollinger middle band,band width,bollinger high gradient,bollinger low gradient,bollinger band percentage
Date,,,,,,,,,,,
2017-01-03,28.950001,29.082500,28.690001,29.037500,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-04,28.962500,29.127501,28.937500,29.004999,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-05,28.980000,29.215000,28.952499,29.152500,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-06,29.195000,29.540001,29.117500,29.477501,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-09,29.487499,29.857500,29.485001,29.747499,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2022-08-09,164.020004,165.820007,163.250000,164.919998,169.935293,143.512707,156.724000,16.859311,0.956165,1.184335,18.411322
2022-08-10,167.679993,169.339996,166.899994,169.240005,171.139772,144.683227,157.911500,16.754033,1.113024,1.075477,18.285841
2022-08-11,170.059998,170.990005,168.190002,168.490005,172.161340,145.663660,158.912500,16.674384,1.330597,0.766904,18.191003


In [8]:
#plotting bollinger band percentage against bollinger high band gradient
#this cell may not be useful
def plot_bandpercent_highgrad(df):
    plt.clf()
    percentage_array = df["bollinger band percentage"].tolist()
    count = 0
    gradient_array = df["bollinger high gradient"].tolist()
    plt.title("band p vs. high grad")
    plt.xlabel("gradient")
    plt.scatter(gradient_array,percentage_array,c="blue")


In [9]:
#plotting gradients and band percentages
def plot_gradients_percentage_3d(df):
    plt.clf()
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    x = df["bollinger low gradient"]
    y = df["bollinger high gradient"]
    z = df["bollinger band percentage"]
    ax.scatter(x, y, z, c=z, cmap='viridis', linewidth=0.5)

In [10]:
#extracts market condition from the rows
#band percentage < %10 --> equilibrium, if not Close > middle --> bull, if not Close < middle --> bear
def extract_market_condition(row):
    if(row["bollinger band percentage"] < 10):
        return 0
    elif(row["Close"] > row["bollinger middle band"]):
        return 1
    else:
        return -1
#applying condition function to the dataframe
def add_condition_df(df):
    df["condition"] = df.apply(lambda row: extract_market_condition(row), axis=1)

In [11]:
#adding the candle percentage return into the dataframe
def add_candle_return_df(df):
    df["candle return"] = (df["Close"] - df["Open"])/df["Open"]*100


In [12]:
#plotting the price closes versus the market condition
def plot_close_condition(df):
    plt.clf()
    fig, axs = plt.subplots(2)
    fig.suptitle('price close vs. condition')
    axs[0].plot(df["Close"])
    axs[1].plot(df["condition"])

new ideas:
<li>
plot percentage return of bullish/bearish periods
</li>
<li>
plot candles of bullish/bearish periods
</li>
to-do:
<li>
try different data: btc on different time frames
</li>


In [13]:
#plotting candle percentage and market condition in bar chart
def bar_plot_condition(row):
    plt.clf()
    if(row["condition"] == 1):
        plt.bar(row["Date"],row["candle return"],color="green")
    elif(row["condition"] == 0):
        plt.bar(row["Date"],row["candle return"],color="blue")
    else:
        plt.bar(row["Date"],row["candle return"],color="red")

def plot_candle_condition(df_csv):
    df_csv.apply(lambda row: bar_plot_condition(row),axis=1)

In [18]:
#df_new = pd.read_csv("test.csv")
#df_new
#df_new.apply(lambda row: bar_plot_condition(row), axis=1)
df.to_csv("test.csv")